pyspark --packages io.delta:delta-core_2.12:1.0.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"

In [2]:
import pyspark

In [3]:
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

In [5]:
display(data)

DataFrame[id: bigint]

In [6]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  3|
|  4|
|  0|
|  2|
|  1|
+---+



In [7]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")

In [8]:
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  8|
|  9|
|  6|
|  5|
|  7|
+---+



In [9]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  8|
|  2|
| 18|
|  4|
| 14|
| 19|
|  7|
|  6|
| 13|
|  0|
| 15|
| 10|
| 17|
| 11|
| 16|
| 12|
|  9|
|  5|
|  3|
|  1|
+---+



In [22]:
df = spark.read.format("delta").option("versionAsOf", 4).load("/tmp/delta-table")
df.show()

+---+
| id|
+---+
|  8|
|  2|
| 18|
|  4|
| 14|
| 19|
|  7|
|  6|
| 13|
|  0|
| 15|
| 10|
| 17|
| 11|
| 16|
| 12|
|  9|
|  5|
|  3|
|  1|
+---+



In [12]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table")